In order to run this notebook, the following environment and packages had to be installed:

$ conda create -n v-env python=3.8

activate v-env

conda activate v-env

conda config --set channel_priority false

conda install -c conda-forge prophet

pip install salesforce-merlion

pip install pingouin

pip install PyYAML

pip install SQLAlchemy

pip install ipykernel

pip install jinja2

pip install sfc-merlion[plot]

pip install -e ts_datasets/

conda install -e ts_datasets/

conda install -c conda-forge lightgbm

conda activate v-env

pip install -e ts_datasets/

pip intall -e ./Merlion/ts_dataset/

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm

#from ts_datasets.anomaly import NAB
#from ts_datasets.forecast import M4

#from merlion.plot import plot_anoms
from merlion.utils import TimeSeries
from merlion.utils.time_series import TimeSeries

from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample
from merlion.transform.moving_average import DifferenceTransform

from merlion.models.defaults import DefaultDetectorConfig, DefaultDetector
#from merlion.models.anomaly.isolation_forest import IsolationForest, IsolationForestConfig
#from merlion.models.anomaly.windstats import WindStats, WindStatsConfig
#from merlion.models.anomaly.forecast_based.prophet import ProphetDetector, ProphetDetectorConfig

from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig

#from merlion.models.ensemble.anomaly import DetectorEnsemble, DetectorEnsembleConfig
from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from merlion.models.ensemble.combine import Mean, ModelSelector
from merlion.models.ensemble.forecast import ForecasterEnsemble, ForecasterEnsembleConfig

#from merlion.evaluate.anomaly import TSADMetric
from merlion.post_process.threshold import AggregateAlarms
from merlion.evaluate.forecast import ForecastMetric

from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector

# With Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

model3 = ModelFactory.create("ForecasterEnsemble", models=[model1, model2],
                             combiner=ModelSelector(metric=ForecastMetric.sMAPE))

for model in [model1, model2, model3]:
    print(f"Training {type(model).__name__}...")
    train_pred, train_stderr = model.train(train_data)

In [ ]:
target = TimeSeries.from_pd(tesla_mv)
target = target.univariates['Close'][(-test_size-1):(-test_size-1+max_forecast_steps)].to_ts()

In [ ]:
from merlion.evaluate.forecast import ForecastMetric

for model in [model1, model2, model3]:
    forecast, stderr = model.forecast(target.time_stamps)
    rmse = ForecastMetric.RMSE.value(ground_truth=target, predict=forecast)
    smape = ForecastMetric.sMAPE.value(ground_truth=target, predict=forecast)
    mae = ForecastMetric.MAE.value(ground_truth=target, predict=forecast)
    
    print(f"{type(model).__name__}")
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")

# No Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop(['target', 'Daily Average Tone'], axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
target_seq_index = 0
max_forecast_steps = 5
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

model3 = ModelFactory.create("ForecasterEnsemble", models=[model1, model2],
                             combiner=ModelSelector(metric=ForecastMetric.sMAPE))

for model in [model1, model2, model3]:
    print(f"Training {type(model).__name__}...")
    train_pred, train_stderr = model.train(train_data)

In [ ]:
target = TimeSeries.from_pd(tesla_mv)
target = target.univariates['Close'][(-test_size-1):(-test_size-1+max_forecast_steps)].to_ts()

In [ ]:
for model in [model1, model2, model3]:
    forecast, stderr = model.forecast(target.time_stamps)
    rmse = ForecastMetric.RMSE.value(ground_truth=target, predict=forecast)
    smape = ForecastMetric.sMAPE.value(ground_truth=target, predict=forecast)
    mae = ForecastMetric.MAE.value(ground_truth=target, predict=forecast)
    
    print(f"{type(model).__name__}")
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")

# Close and Tone Only

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv = tesla_mv[['Close', 'Daily Average Tone']]

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
target_seq_index = 0
max_forecast_steps = 5 # 5
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

model3 = ModelFactory.create("ForecasterEnsemble", models=[model1, model2],
                             combiner=ModelSelector(metric=ForecastMetric.sMAPE))

for model in [model1, model2, model3]:
    print(f"Training {type(model).__name__}...")
    train_pred, train_stderr = model.train(train_data)

In [ ]:
target = TimeSeries.from_pd(tesla_mv)
target = target.univariates['Close'][(-test_size-1):(-test_size-1+max_forecast_steps)].to_ts()

In [ ]:
for model in [model1, model2, model3]:
    forecast, stderr = model.forecast(target.time_stamps)
    rmse = ForecastMetric.RMSE.value(ground_truth=target, predict=forecast)
    smape = ForecastMetric.sMAPE.value(ground_truth=target, predict=forecast)
    mae = ForecastMetric.MAE.value(ground_truth=target, predict=forecast)
    
    print(f"{type(model).__name__}")
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")

In [ ]:
forecast

In [ ]:
target

# 5 Day Horizon With Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv

In [ ]:
#tesla_mv = tesla_mv[['Close', 'Daily Average Tone']]

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=target, predict=forecast, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {smape:.4f}")
    print()

# 5 Day Horizon No Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop(['target', 'Daily Average Tone'], axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=target, predict=forecast, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {smape:.4f}")
    print()

# 3 Day Horizon Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=target, predict=forecast, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {smape:.4f}")
    print()

# 3 Day No Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop(['target', 'Daily Average Tone'], axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=target, predict=forecast, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {smape:.4f}")
    print()

# 1 Day Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=target, predict=forecast, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {smape:.4f}")
    print()

# 1 Day No Tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop(['target', 'Daily Average Tone'], axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 3
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

# 1 Day Close and Tone 

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv = tesla_mv[['Close', 'Daily Average Tone']]

In [ ]:
tesla_mv

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

# 1 Day Close Only

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

In [ ]:
tesla_mv = tesla_mv[['Close']]

In [ ]:
tesla_mv

In [ ]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

In [ ]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

# 3 day close and tone

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

tesla_mv = tesla_mv[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

# 3 day close only

In [ ]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

tesla_mv = tesla_mv[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

# 5 day close and tone

In [4]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

tesla_mv = tesla_mv[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:23<00:00, 216695.24it/s]


RMSE:  3.3713
sMAPE: 4.7436
MAE:   2.9712

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2383351.50it/s]


RMSE:  3.1129
sMAPE: 4.3883
MAE:   2.7307



In [202]:
test_data

                Close  Daily Average Tone
2017-06-03  71.634003           -1.347826
2017-06-04  74.166000           -0.322917
2017-06-05  70.510002           -2.159091
2017-06-06  72.444000           -2.112069
2017-06-07  69.501999           -3.732057
...               ...                 ...
2017-12-25  55.212002            0.187500
2017-12-26  53.683998           -1.000000
2017-12-27  53.540001           -0.137931
2017-12-28  53.276001           -0.650000
2017-12-29  54.672001           -0.166667

[210 rows x 2 columns]

In [296]:
real_date = tesla['Date'][-210:]

In [297]:
real_date = real_date.reset_index()

In [298]:
real_date.drop('index', axis=1, inplace=True)

In [299]:
real_date

,Date
0,2018-06-15
1,2018-06-18
2,2018-06-19
3,2018-06-20
4,2018-06-21
...,...
205,2019-04-10
206,2019-04-11
207,2019-04-12
208,2019-04-15


In [300]:
tesla_mv2 = tesla_mv[-210:]

In [301]:
tesla_mv2.reset_index(inplace=True)

In [302]:
tesla_mv2.drop('Date',inplace=True, axis=1)

/home/msds2022/mpingol/.conda/envs/v-env/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [303]:
tesla_mv2['Date'] = real_date['Date']

/tmp/ipykernel_16854/4260535548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_mv2['Date'] = real_date['Date']


In [304]:
tesla_mv2

,Close,Date
0,71.634003,2018-06-15
1,74.166000,2018-06-18
2,70.510002,2018-06-19
3,72.444000,2018-06-20
4,69.501999,2018-06-21
...,...,...
205,55.212002,2019-04-10
206,53.683998,2019-04-11
207,53.540001,2019-04-12
208,53.276001,2019-04-15


In [305]:
pred_list = []
for i in test_pred:
    pred_list.append(pd.Series(i))

In [306]:
len(pred_list)

210

In [307]:
for i in range(len(pred_list)):
    tesla_mv2.loc[tesla_mv2.index[i], 'Pred'] = pred_list[i][(len(pred_list[i])-1)][1][0]

/home/msds2022/mpingol/.conda/envs/v-env/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/msds2022/mpingol/.conda/envs/v-env/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [308]:
tesla_mv2

,Close,Date,Pred
0,71.634003,2018-06-15,71.382574
1,74.166000,2018-06-18,71.392613
2,70.510002,2018-06-19,74.033633
3,72.444000,2018-06-20,70.238936
4,69.501999,2018-06-21,72.492763
...,...,...,...
205,55.212002,2019-04-10,54.569974
206,53.683998,2019-04-11,55.339125
207,53.540001,2019-04-12,53.691771
208,53.276001,2019-04-15,53.595438


In [223]:
test_pred

[                Close
 2017-06-03  71.606169
 2017-06-04  71.540072
 2017-06-05  71.473274
 2017-06-06  71.419272
 2017-06-07  71.382574,
                 Close
 2017-06-04  71.569281
 2017-06-05  71.502413
 2017-06-06  71.448175
 2017-06-07  71.411189
 2017-06-08  71.392613,
                 Close
 2017-06-05  74.227333
 2017-06-06  74.166653
 2017-06-07  74.107652
 2017-06-08  74.062234
 2017-06-09  74.033633,
                 Close
 2017-06-06  70.265797
 2017-06-07  70.216018
 2017-06-08  70.202115
 2017-06-09  70.211939
 2017-06-10  70.238936,
                 Close
 2017-06-07  72.501760
 2017-06-08  72.482453
 2017-06-09  72.473810
 2017-06-10  72.478292
 2017-06-11  72.492763,
                 Close
 2017-06-08  69.334593
 2017-06-09  69.333393
 2017-06-10  69.363306
 2017-06-11  69.408785
 2017-06-12  69.460466,
                 Close
 2017-06-09  66.596010
 2017-06-10  66.632395
 2017-06-11  66.699990
 2017-06-12  66.778636
 2017-06-13  66.855669,
                 Close
 201

In [309]:
import pickle

pickle.dump( tesla_mv2, open( "closeonly.p", "wb" ) )

# 5 day close only

In [295]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/tesla_complete.csv')
tesla_mv = tesla.drop('target', axis=1)
tesla_mv['Date'] = pd.date_range(tesla_mv['Date'][0], periods=tesla_mv.shape[0], freq='D')
tesla_mv.set_index('Date', inplace=True)

tesla_mv = tesla_mv[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1841105.43it/s]


RMSE:  3.9592
sMAPE: 5.6749
MAE:   3.5284

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2671442.92it/s]


RMSE:  3.1129
sMAPE: 4.3883
MAE:   2.7307

